### Let's develop a chatbot based on groq llms using langchain and deploy it on haggingface 

In [29]:
import os 
import warnings
from langchain_groq import ChatGroq
from langchain_community.llms import HuggingFaceEndpoint

warnings.filterwarnings("ignore")

In [ ]:
#Test wether api key is there or not
os.environ["HUGGINGFACE_API_TOKEN"]

In [5]:
groq_llm = ChatGroq(
    groq_api_key = os.environ["GROQ_API_KEY"],
    model_name = "llama-3.1-8b-instant", #Check groq website for other llm ids
    temperature = 0.6 
)

In [7]:
#test
test_prompt = "Hello, do you hear me ?"
response = groq_llm.invoke(test_prompt)
print (response.content)

Hello. I can see that you've initiated a conversation. I'm here to assist you with any questions or topics you'd like to discuss. How can I help you today?


In [10]:
#test 2
groq_llm.predict(test_prompt)

'Hello, yes I hear you. Is there something I can help you with or would you like to chat?'

In [37]:
flan_t5_llm = HuggingFaceEndpoint(huggingfacehub_api_token = os.environ["HUGGINGFACE_API_TOKEN"],
                                repo_id="google/flan-t5-large", 
                                task = 'text-generation',
                                temperature = 0)

In [ ]:
flan_t5_llm.invoke("hello")

### Prompt Templates 

In [45]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [43]:
prompt_template = PromptTemplate(input_variables = ["word"],
                                 template = "What are 5 synonyms of word {word}")
prompt_template.format(word = "impossible")

'What are 5 synonyms of word impossible'

In [48]:
chain = LLMChain(llm = groq_llm, prompt = prompt_template)
synonyme_response = chain.run("impossible")
print (synonyme_response)

Here are 5 synonyms for the word "impossible":

1. Unfeasible
2. Inconceivable
3. Unthinkable
4. Unachievable
5. Unattainable


In [49]:
print (chain.run("possible"))

Here are 5 synonyms of the word "possible":

1. Feasible
2. Probable
3. Viable
4. Plausible
5. Likely


the 